In [2]:
from glove import Corpus, Glove
import numpy as np
from os import path, makedirs
from pickle import dump, load

Загрузка данных

In [4]:
with open(path.join('corpus', 'model_data.pickle'), 'rb') as f:
    model_data = load(f)

Обучение

In [5]:
corpus = Corpus()
corpus.fit(model_data, window=7)

In [6]:
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [7]:
glove.add_dictionary(corpus.dictionary)

In [8]:
g_vocab = glove.dictionary

In [12]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [13]:
glove.most_similar(morph.parse('абу')[0].normal_form, number = 10)

[('спасибо', 0.92330191281519225),
 ('бро', 0.90343762918547654),
 ('наводка', 0.88255063046008742),
 ('заранее', 0.86424484902923249),
 ('перехотелосьнуть', 0.85473405430210925),
 ('анончик', 0.85438517265997838),
 ('антоша', 0.8536180602273532),
 ('пояснение', 0.84962745761518721),
 ('cпасибо', 0.8482939899336468)]

In [9]:
subdir = 'models'
if not path.exists(subdir):
    makedirs(subdir)
file_path = path.join(subdir, '2ch_glove_all_lem_100')

with open(file_path, 'wb') as fp:
    dump(glove, fp)